In [1]:
import sklearn
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np
import pickle
import os

Load the Data

In [6]:
def unPickleData(filename):
  with open(filename,'rb') as f: arr = pickle.load(f)
  return arr

def getDataPath(dirname,filename):
  return os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)),"processed_data", dirname,filename)

x_data = unPickleData(getDataPath("posts","X_posts.pkl"))
y_data = unPickleData(getDataPath("posts","Y_posts.pkl"))

In [10]:
X, y = [], []
Class0_max_count = 20000
for i in range(len(x_data)):
    if y_data[i] == 0: 
        if Class0_max_count > 0:
            X.append(x_data[i])
            y.append(y_data[i])
            if i%1000 == 0: print(i, end=" ")
            Class0_max_count -= 1 
    else:
        X.append(x_data[i])
        y.append(y_data[i])

np_y = np.array(y)
print("class 0:", len(np_y[np_y==0]))
print("class 1:", len(np_y[np_y==1]))

0 2000 3000 4000 5000 6000 7000 8000 9000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.6 , random_state= 42)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5 , random_state= 42)

SVM trainig & Hyperparameter Tuning

In [13]:
parameters = {'C':[0.001,0.01,0.1, 1]}
linearSVM = SVC(kernel="linear")
linearClassifier = GridSearchCV(linearSVM, parameters, scoring = ['f1_micro','accuracy'], cv=5, refit='f1_micro')
linearClassifier.fit(X_train, y_train)

bestLinearClassifier = linearClassifier.best_estimator_

print("For Linear SVM:")
print(f"Best Estimator: {linearClassifier.best_estimator_}")
print(f"F-measure = {linearClassifier.best_score_}")
print(f"Accuracy = {max(linearClassifier.cv_results_['mean_test_f1_micro'])}")

Testing

In [ ]:
y_pred = bestLinearClassifier.predict(X_test)

print(sklearn.metrics.classification_report(y_test,y_pred))
print("Accuracy: ", sklearn.metrics.accuracy_score(y_test, y_pred))
print(f"f-measure = {sklearn.metrics.f1_score(y_test, y_pred, average='micro')}")

Saving the model

In [ ]:
def getSavedModelPath(dirname,filename):
  return os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)),"savedModels", dirname,filename)

with open(getSavedModelPath("posts","SVM.pkl"), 'wb') as f:
    pickle.dump(bestLinearClassifier,f)

Learning Curve

In [2]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

train_size, train_score, test_scores = learning_curve(bestLinearClassifier, X_train, y_train, cv = 10, scoring='accuracy', n_jobs=1, train_sizes=np.linspace(0.01, 1, 50), verbose=1)
mean_train = np.mean(train_score , axis = 1)
mean_test = np.mean(test_scores , axis = 1)
plt.plot(train_size, 1-mean_train, label='Training score')
plt.plot(train_size, 1-mean_test, label='Cross-validation score')
plt.xlabel('Training Size')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend()
plt.show()

NameError: name 'linearClassifier' is not defined